In [1]:
import sqlite3 as lite
import pandas as pd

In [2]:
con = lite.connect('Parkingproject.db')
cur = con.cursor()

print('db and cursor created')

db and cursor created


# TABLE CREATE

In [3]:
sql_file = open('createtables.sql')
sql_parking = sql_file.read()
cur.executescript(sql_parking)
print('Tables created')

Tables created


# INSERT DATA

In [4]:
sql_file_insert=open('INSERT.sql')
sql_parking_data=sql_file_insert.read()
cur.executescript(sql_parking_data)
print('Data Inserted')

Data Inserted


# QUERY

In [87]:
#Find the name of owner of cars and model using plate number
plate_number=input('ENTER PLATE NUMBER: ')

q1='''
select o.owner_name,v.vehicle_model
from vehicle as v,owners as o 
where o.owner_wing=v.owner_wing and
o.flat_number=v.flat_number and
v.plate_number='{}'
'''.format(plate_number)
table=pd.read_sql_query(q1,con)
table

ENTER PLATE NUMBER: MH 02 ER 2598


,owner_name,vehicle_model
0,GAURAV BHATIA,MCLAREN P1


# Find how many people visited a resident in buiding at a time using owners name

q2='''
select v1.visitor_name,v2.no_of_visitors as No_Of_Visitors 

from owners as o, visiting as v1,visitors as v2

where o.owner_wing=v1.owner_wing and o.flat_number=v1.flat_number and
v1.visitor_name=v2.visitor_name and v1.visitor_parking_number=v2.visitor_parking_number
and o.owner_name='MUKESH AMBANI'

group by v1.flat_number
'''
table=pd.read_sql_query(q2,con)
table

In [6]:
#Find how many people visited a resident in buiding at a time using owners name
name=input('Enter owners name: ')
q2='''
select o.owner_name,v.visitor_name,v.no_of_visitors as No_Of_Visitors
from owners as o,(select v2.flat_number,v2.owner_wing,v2.visitor_name,v1.no_of_visitors
                 from visitors as v1,visiting as v2 
                 where v1.visitor_name=v2.visitor_name and v1.visitor_parking_number=v2.visitor_parking_number) as v
where o.owner_name='{}' and
o.flat_number=v.flat_number and
o.owner_wing=v.owner_wing

'''.format(name)
table=pd.read_sql_query(q2,con)
table

Enter owners name: MUKESH AMBANI


,owner_name,visitor_name,No_Of_Visitors
0,MUKESH AMBANI,Roger,5


In [7]:
#Find all the residents who own two wheeler and four wheeler and also find how many 

q3='''
select o.owner_name,v.vehicle_type,count(v.vehicle_type) as number_of_vehicle
from vehicle as v,owners as o
where o.owner_wing=v.owner_wing and o.flat_number=v.flat_number and vehicle_type <> 'ONE WHEELER'
group by vehicle_type,owner_name

'''
table=pd.read_sql_query(q3,con)
table

,owner_name,vehicle_type,number_of_vehicle
0,ANIL AMBANI,FOUR WHEELER,1
1,GAURAV BHATIA,FOUR WHEELER,2
2,MUKESH AMBANI,FOUR WHEELER,2
3,PAVAN KHERA,FOUR WHEELER,1
4,PRIYANKA GANDHI,FOUR WHEELER,1
5,SAMBIT PATRA,FOUR WHEELER,2
6,SUDHANSHU CHATURVEDI,FOUR WHEELER,1
7,MUKESH AMBANI,TWO WHEELER,1
8,SUDHANSHU CHATURVEDI,TWO WHEELER,1


In [8]:
# find all vehicle models
q4='''
select distinct(vehicle_model),vehicle_type 
from vehicle 
order by vehicle_type
'''
table=pd.read_sql_query(q4,con)
table

,vehicle_model,vehicle_type
0,MARUTI 800,FOUR WHEELER
1,ALTO,FOUR WHEELER
2,MCLAREN P1,FOUR WHEELER
3,SKYLINE GTR,FOUR WHEELER
4,SHELBY MUSTANG GT,FOUR WHEELER
5,ROLLS-ROYCE GHOST,FOUR WHEELER
6,KOENIGSEGG AGERA R,FOUR WHEELER
7,ASTON MARTIN DB11,FOUR WHEELER
8,PORSHE 911,FOUR WHEELER
9,LAMBORGHINI ELEMENTO,FOUR WHEELER


In [9]:
# find the type and name of vehicles owned by Anil Ambani

q5='''
select v.vehicle_type,v.vehicle_model
from vehicle as v, owners as o
where o.owner_wing=v.owner_wing and
o.flat_number=v.flat_number and
o.owner_name='MUKESH AMBANI'
'''
table=pd.read_sql_query(q5,con)
table


,vehicle_type,vehicle_model
0,FOUR WHEELER,ROLLS-ROYCE GHOST
1,TWO WHEELER,DUCATTI XDIAVEL
2,FOUR WHEELER,KOENIGSEGG AGERA R


In [10]:
# find all residents with four as well as two wheeler vehicles

q6='''
select distinct(o1.owner_name)
from owners as o1,owners as o2,vehicle as v1,vehicle as v2
where o1.owner_wing=o2.owner_wing and
o1.flat_number=o2.flat_number and
o1.owner_wing=v1.owner_wing and
o2.flat_number=v2.flat_number and
o1.owner_name=o2.owner_name and
v1.flat_number=v2.flat_number and
v1.owner_wing=v2.owner_wing and
v1.vehicle_type='TWO WHEELER' and
v2.vehicle_type='FOUR WHEELER'
'''
table=pd.read_sql_query(q6,con)
table

,owner_name
0,SUDHANSHU CHATURVEDI
1,MUKESH AMBANI


In [11]:
# find all the owners and their vehicle details using join operation ordered by owner name and then vehicle_name
q7='''
select *
from owners as o natural left outer join vehicle as v
order by owner_name,vehicle_model
'''
table=pd.read_sql_query(q7,con)
table

,owner_name,owner_wing,flat_number,number_of_slots,vehicle_type,plate_number,vehicle_model
0,ANIL AMBANI,B,104,1,FOUR WHEELER,MH 47 AC 8852,ASTON MARTIN DB11
1,GAURAV BHATIA,B,503,2,FOUR WHEELER,MH 02 ER 2598,MCLAREN P1
2,GAURAV BHATIA,B,503,2,FOUR WHEELER,MH 02 ER 9536,SKYLINE GTR
3,MUKESH AMBANI,A,701,3,TWO WHEELER,MH 47 A 9856,DUCATTI XDIAVEL
4,MUKESH AMBANI,A,701,3,FOUR WHEELER,MH 47 A 9854,KOENIGSEGG AGERA R
5,MUKESH AMBANI,A,701,3,FOUR WHEELER,MH 47 A 9852,ROLLS-ROYCE GHOST
6,PAVAN KHERA,A,101,1,FOUR WHEELER,MH 47 A 1638,MARUTI 800
7,PRIYANKA GANDHI,A,102,1,FOUR WHEELER,MH 27 D 1792,ALTO
8,RAHUL GANDHI,D,101,1,ONE WHEELER,DU 09 MB 9999,UNICYCLE
9,SAMBIT PATRA,D,702,2,FOUR WHEELER,MH 02 AY 9848,LAMBORGHINI ELEMENTO


In [12]:
# find all owners who have max parking slots

q8='''
select distinct(o.owner_name),number_of_slots
from owners as o , parking_slot as ps
where o.owner_wing=ps.owner_wing and o.flat_number=ps.flat_number
group by parking_number
having number_of_slots = (select max(number_of_slots) from owners)

'''
table=pd.read_sql_query(q8,con)
table

,owner_name,number_of_slots
0,MUKESH AMBANI,3


In [13]:
# find all owners of cars sgarting with letter s

q9='''
select v.vehicle_model from vehicle as v where vehicle_model like 's%'
'''
table=pd.read_sql_query(q9,con)
table

,vehicle_model
0,SKYLINE GTR
1,SPLENDOR
2,SHELBY MUSTANG GT


In [14]:
# Find all vehicles owned by using owner_name

owner_name=input('ENTER OWNER NAME: ')

q10='''
select o.owner_name,v.vehicle_model,v.plate_number 
from vehicle as v ,owners as o
where o.owner_wing=v.owner_wing and
o.flat_number=v.flat_number and
o.owner_name='{}'
'''.format(owner_name)

table=pd.read_sql_query(q10,con)
table

ENTER OWNER NAME: MUKESH AMBANI


,owner_name,vehicle_model,plate_number
0,MUKESH AMBANI,ROLLS-ROYCE GHOST,MH 47 A 9852
1,MUKESH AMBANI,DUCATTI XDIAVEL,MH 47 A 9856
2,MUKESH AMBANI,KOENIGSEGG AGERA R,MH 47 A 9854


In [90]:
#update NO of visitors
name=input('ENTER NAME: ')
no_of_visitors=int(input('ENTER no of visitors: '))

q11='''
update visitors
set no_of_visitors='{}'
where visitor_name='{}'
'''.format(no_of_visitors,name)
cur.execute(q11)
q111='''
select * from visitors
'''
table=pd.read_sql_query(q111,con)
table


ENTER NAME: Katrina
ENTER no of visitors: 1


,visitor_name,visitor_vehicle_type,visitor_parking_number,visitor_in,visitor_out,no_of_visitors
0,Rohan,TWO WHEELER,101,13:12:25,16:23:14,3
1,Roger,THREE WHEELER,102,17:41:23,20:20:22,5
2,Thomas,TWO WHEELER,103,12:45:50,23:05:52,1
3,Sheena,TWO WHEELER,104,21:15:26,02:45:05,4
4,Raveena,FOUR WHEELER,105,21:15:26,02:45:05,2
5,Katrina,TWO WHEELER,106,21:16:26,03:48:05,1


In [20]:
#find the total number of parking spots using wing A to D
owner_wing=input('Enter wing: ')
q12='''
select count(parking_number) as total
from parking_slot
where owner_wing='{}'
'''.format(owner_wing)

table=pd.read_sql_query(q12,con)
table

Enter wing: B


,total
0,3


In [84]:
#calculate and show the amount each owner has to pay if each 4 wheeler has parking cost 400rs and 2 Wheeler has 200rs
#and onewheeler is at home
q131='''

select owner_name,((400*Four_wheelers)+(200*Two_wheelers)) as AMOUNT
from owners as o 
natural inner join 
(select o.owner_name,count(v.plate_number) as Four_wheelers 
from vehicle as v natural inner join owners as o where vehicle_type='FOUR WHEELER' group by o.owner_name) as v1 
natural inner join 
(select o.owner_name,count(v.plate_number) as Two_wheelers 
from vehicle as v natural inner join owners as o where vehicle_type='TWO WHEELER' group by o.owner_name) as v2

'''

table=pd.read_sql_query(q131,con)
table

,owner_name,AMOUNT
0,MUKESH AMBANI,1000
1,SUDHANSHU CHATURVEDI,600


In [85]:
q132='''

select o.owner_name,(400*Four_wheelers) as AMOUNT
from owners as o 
natural inner join 
(select o.owner_name,count(v.plate_number) as Four_wheelers 
from vehicle as v natural inner join owners as o 
where vehicle_type='FOUR WHEELER' 
group by o.owner_name) as v1 
where owner_name 
not in 
(select owner_name 
from owners as o 
natural inner join
(select o.owner_name,count(v.plate_number) as Four_wheelers 
from vehicle as v natural inner join owners as o where vehicle_type='FOUR WHEELER' group by o.owner_name) as v1 
natural inner join 
(select o.owner_name,count(v.plate_number) as Two_wheelers 
from vehicle as v natural inner join owners as o where vehicle_type='TWO WHEELER' group by o.owner_name) as v2)

'''

table=pd.read_sql_query(q132,con)
table

,owner_name,AMOUNT
0,PAVAN KHERA,400
1,PRIYANKA GANDHI,400
2,GAURAV BHATIA,800
3,ANIL AMBANI,400
4,SAMBIT PATRA,800


In [86]:
q133='''

select o.owner_name,(Two_wheelers*200) as AMOUNT
from owners as o 
natural inner join
(select o.owner_name,count(v.plate_number) as Two_wheelers 
from vehicle as v natural inner join owners as o where vehicle_type='TWO WHEELER' group by o.owner_name) as v1 
where owner_name 
not in 
(select owner_name 
from owners as o 
natural inner join 
(select o.owner_name,count(v.plate_number) as Four_wheelers 
from vehicle as v natural inner join owners as o where vehicle_type='FOUR WHEELER' group by o.owner_name) as v1 
natural inner join 
(select o.owner_name,count(v.plate_number) as Two_wheelers 
from vehicle as v natural inner join owners as o where vehicle_type='TWO WHEELER' group by o.owner_name) as v2)

'''
table=pd.read_sql_query(q133,con)
table

,owner_name,AMOUNT
